In [ ]:
from modules.load_data.get_financial import get_data, generate_data

try:
    data = get_data(0)
    # print(data)
except Exception as e:
    data = generate_data(start_date, end_date, 0)

# print(data)
x, y = data, data[:, -1].astype(np.float32)
scaler = y[:int(len(x) * config.density)].astype(np.float32)
y = (y - np.mean(scaler)) / np.std(scaler)
# print(scaler.shape)
x[:, -1] = x[:, -1].astype(np.float32)
temp = x[:, -1].astype(np.float32)
x[:, -1] = (temp - np.mean(scaler)) / np.std(scaler)
x = x.astype(np.float32)
y = y.astype(np.float32)
X_window, y_window = create_window_dataset(x, y, config.seq_len, config.pred_len)



In [3]:
import numpy as np
import optuna

# 计算 MAE
def calculate_mae(y, y_hat):
    return np.mean(np.abs(y - y_hat))

# 训练并计算 MAE
def train_and_evaluate(params):
    size = 100
    y = np.random.rand(size) * 10  # 真实值
    y_hat = np.random.rand(size) * 10  # 预测值
    mae = calculate_mae(y, y_hat)
    return mae

# **定义超参数搜索空间**
search_space = {
    'rank': [128],
    'num_layers': [1, 2, 3, 4],
    'dataset': ['weather'],
    'att': ['mla', 'self', 'external', 'gqa', 'mqa'],
    'norm': ['rms', 'layer', 'batch'],
    'ffn': ['ffn', 'moe']
}

# **固定某些参数（如果需要）**
fixed_params = {
    'dataset': 'weather'  # 例如我们只优化其他参数，dataset 固定为 'weather'
}

# **使用 TPE 采样器**
base_sampler = optuna.samplers.TPESampler()

# **创建 PartialFixedSampler（固定部分参数）**
sampler = optuna.samplers.PartialFixedSampler(fixed_params, base_sampler)

def objective(trial):
    # **动态生成超参数（非固定参数）**
    params = {key: trial.suggest_categorical(key, values) for key, values in search_space.items() if key not in fixed_params}

    # **添加固定参数**
    params.update(fixed_params)

    return train_and_evaluate(params)

# **创建 study 并运行优化**
study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=15)  #

# **输出结果**
print("Best Parameters:", study.best_params)
print("Best MAE:", study.best_value)

/var/folders/x5/s_l7bmrn7wndpynfr_mrrdg40000gn/T/ipykernel_77843/2300174442.py:35: ExperimentalWarning: PartialFixedSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler = optuna.samplers.PartialFixedSampler(fixed_params, base_sampler)
[I 2025-03-16 15:46:05,778] A new study created in memory with name: no-name-fe174e23-f5cd-4290-b4eb-77a6d83eea6b
[I 2025-03-16 15:46:05,779] Trial 0 finished with value: 3.489385154427884 and parameters: {'rank': 128, 'num_layers': 3, 'att': 'mqa', 'norm': 'rms', 'ffn': 'moe'}. Best is trial 0 with value: 3.489385154427884.
[I 2025-03-16 15:46:05,780] Trial 1 finished with value: 3.3493881203062417 and parameters: {'rank': 128, 'num_layers': 4, 'att': 'mla', 'norm': 'batch', 'ffn': 'ffn'}. Best is trial 1 with value: 3.3493881203062417.
[I 2025-03-16 15:46:05,781] Trial 2 finished with value: 3.3236907552820054 and parameters: {'rank': 128, 'num_layers': 2, 'att': 'external', 'norm': 'rms', 'ffn': 'moe'}. Bes

Best Parameters: {'rank': 128, 'num_layers': 2, 'att': 'gqa', 'norm': 'layer', 'ffn': 'ffn'}
Best MAE: 2.912417094375815
